##  **What this notebook does:**
- Starts from the **mushrooms dataset** and integer-encodes all features

- Randomly injects NaNs into selected target columns (y) to simulate missing labels

- Searches for the best feature combinations (x) to predict each y by evaluating KNN accuracy

- Uses those top x features to impute the NaNs in y with KNN

- Implements the full pipeline twice to compare NumPy vs PyTorch, with PyTorch able to use CUDA when available for vectorized speedups
- Helps us compare NumPy vs Torch implementations in terms of speed and results.

# **Numpy version**

## Measuring Runtime
We measure the **total runtime** of the entire pipeline, from the very beginning (data preparation and NaN injection) through  
finding the **best features** and finally imputing the missing values (NaNs) with KNN.  

This end-to-end timing allows us to compare execution speed between different implementations (NumPy vs PyTorch).


In [ ]:
# Measuring Runtime
import time
start_time = time.time()

## Import Required Libraries (NumPy Implementation)

In [ ]:
# Import Required Libraries (NumPy Implementation)
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from scipy.stats import mode
from time import perf_counter
import itertools

## Load and Encode Dataset

At first, we load the **mushrooms dataset** from `mushrooms.csv`.  
Since all features are categorical (strings), we apply **integer encoding** to convert them into numeric values suitable for KNN.  


In [ ]:
# Load the dataset (expects mushrooms.csv to be present)
df = pd.read_csv("mushrooms.csv")

# Integer-encode each categorical column; keep mappings if needed
enc_mappings = {}
for col in df.columns:
    codes, uniques = pd.factorize(df[col], sort=False)
    df[col] = codes
    enc_mappings[col] = dict(zip(uniques, range(len(uniques))))

df.head()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
2,1,1,0,2,0,2,0,0,1,1,...,0,0,0,0,0,0,0,1,1,2
3,0,0,1,2,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,3,1,3,0,1,1,0,...,0,0,0,0,0,0,1,1,2,1


## Prepare Data and Inject Missing Values

We first convert the DataFrame into a NumPy array (using `float32` so NaNs are supported).  
Then, we randomly inject **10% NaNs** into selected target columns (`cap-color`, `ring-number`, `spore-print-color`) to simulate missing labels.  
Finally, we do a quick check to confirm the number of NaNs injected per column.


In [ ]:
# Convert DataFrame to numpy array (float for NaN support)
data = df.values.astype(np.float32)
n_rows, n_cols = data.shape

# Randomly inject NaNs in target columns
np.random.seed(42)
target_cols = ['cap-color', 'ring-number', 'spore-print-color']
target_idx = [df.columns.get_loc(c) for c in target_cols]

nan_frac = 0.10  # 10% NaNs per target column
for j in target_idx:
    idx = np.random.choice(n_rows, size=int(n_rows * nan_frac), replace=False)
    data[idx, j] = np.nan

# Quick check: NaN counts per target
nan_counts = [(c, np.isnan(data[:, df.columns.get_loc(c)]).sum()) for c in target_cols]
print(nan_counts)

[('cap-color', np.int64(812)), ('ring-number', np.int64(812)), ('spore-print-color', np.int64(812))]


## KNN Imputation with NumPy  
Function to fill missing values in a target column using KNN over selected features.


In [ ]:
def knn_impute_numpy(data, y_col_idx, x_col_indices, k=3, batch_size=512):
    """
    Impute missing labels in column y_col_idx using KNN over features x_col_indices.
    data: numpy array [N, D] with NaNs allowed (float dtype). Operates in-place on a copy.
    Returns a new array with NaNs in y_col_idx imputed.
    """
    X = data[:, x_col_indices]  # [N, F]
    y = data[:, y_col_idx]      # [N]

    # Identify missing/non-missing in y
    missing_mask = np.isnan(y)
    keep_mask = ~missing_mask

    # If nothing to impute, return as-is
    if keep_mask.sum() == 0 or missing_mask.sum() == 0:
        return data.copy()

    # Training set (non-missing rows)
    train_X = X[keep_mask]      # [M, F]
    train_y = y[keep_mask].astype(int)

    # Result copy we will fill
    out = data.copy()

    # Handle NaNs in training features: replace with column means
    if np.isnan(train_X).any():
        col_means = np.nanmean(train_X, axis=0)
        inds = np.where(np.isnan(train_X))
        train_X[inds] = np.take(col_means, inds[1])
    else:
        col_means = None

    # Indices to impute
    miss_idx = np.where(missing_mask)[0]

    # Loop in batches to limit memory for cdist
    for start in range(0, len(miss_idx), batch_size):
        end = min(start + batch_size, len(miss_idx))
        idx_batch = miss_idx[start:end]

        test_X = X[idx_batch]

        # Fill NaNs in test_X with training means if present
        if np.isnan(test_X).any():
            if col_means is None:
                col_means = np.nanmean(train_X, axis=0)
            inds = np.where(np.isnan(test_X))
            test_X[inds] = np.take(col_means, inds[1])

        # Pairwise Euclidean distances
        dist = cdist(test_X, train_X, metric="euclidean")  # [B, M]

        # K nearest neighbors
        idxs = np.argpartition(dist, kth=min(k, dist.shape[1]) - 1, axis=1)[:, :k]

        # Neighbor labels and majority vote
        nn_labels = train_y[idxs]  # [B, k]
        preds, _ = mode(nn_labels, axis=1, keepdims=False)

        out[idx_batch, y_col_idx] = preds.astype(np.float32)

    return out


## Best KNN Feature Combos
Best feature combinations ranked by classification accuracy for each target column.


In [ ]:
def knn_accuracy_all_numpy(df, target_cols, k=3, feature_sample_size=10, comb_size=3):
    results = {}

    for y_col in target_cols:
        # rows where y is known
        non_missing = df[df[y_col].notna()]
        if non_missing.shape[0] <= 1:
            print(f"Skipping {y_col}: no known labels after NaN injection.")
            continue

        # candidate features excluding y_col
        x_candidates = [c for c in df.columns if c != y_col]
        x_candidates = x_candidates[:feature_sample_size]

        if len(x_candidates) < comb_size:
            print(f"Skipping {y_col}: not enough features for comb_size={comb_size}.")
            continue

        combos = list(itertools.combinations(x_candidates, comb_size))
        col_rows = []

        for combo in combos:
            combo = list(combo)

            # keep only rows with complete features for this combo
            nm = non_missing[non_missing[combo].notna().all(axis=1)]
            m = nm.shape[0]
            if m <= 1:
                continue

            # data arrays
            X = nm[combo].to_numpy(dtype=np.float32)  # [m, F]
            y = nm[y_col].to_numpy()                  # [m]

            # pairwise squared distances: ||x - x'||^2 using Gram trick
            x2 = np.sum(X * X, axis=1, keepdims=True)        # [m,1]
            dists = x2 - 2.0 * (X @ X.T) + x2.T              # [m,m]
            np.fill_diagonal(dists, np.inf)                  # exclude self

            # k nearest neighbors
            k_eff = min(k, m - 1)
            idx = np.argpartition(dists, kth=k_eff-1, axis=1)[:, :k_eff]  # [m,k]

            # Neighbor labels
            nn_labels = y[idx]  # [m,k]

            # Map labels to compact range
            unique_labels, y_inv = np.unique(y, return_inverse=True)  # y_inv: [m]
            mapped = y_inv[idx]                                       # [m,k]

            # Count votes per row
            L = unique_labels.shape[0]
            counts = np.zeros((m, L), dtype=np.int32)
            # fast row-wise bincount
            for r in range(m):
                counts[r] = np.bincount(mapped[r], minlength=L)

            pred_idx = counts.argmax(axis=1)        # [m]
            preds = unique_labels[pred_idx]         # [m]

            correct = np.sum(preds == y)
            acc = 100.0 * correct / m
            col_rows.append((tuple(combo), float(acc)))

        if col_rows:
            col_rows.sort(key=lambda x: x[1], reverse=True)
            results[y_col] = pd.DataFrame(col_rows, columns=["Feature Combination", "Accuracy (%)"])

    return results


### Example Usage
Choose which feature set size you want to evaluate by editing `comb_size`.  
The code includes examples for **3, 4, and 5 features**, simply **uncomment the block you need** and leave the others commented out.


In [ ]:
# 10 best 3-feature combos
# results = knn_accuracy_all_numpy(df, target_cols, k=3, feature_sample_size=10, comb_size=3)
# for y_col, res_df in results.items():
#     print(f"\n📌 Top 10 for {y_col} with 3 features")
#     print(res_df.head(10))

# # 10 best 4-feature combos
# results = knn_accuracy_all_numpy(df, target_cols, k=3, feature_sample_size=10, comb_size=4)
# for y_col, res_df in results.items():
#     print(f"\n📌 Top 10 for {y_col} with 4 features")
#     print(res_df.head(10))

# # 10 best 5-feature combos
results = knn_accuracy_all_numpy(df, target_cols, k=3, feature_sample_size=5, comb_size=5)
for y_col, res_df in results.items():
    print(f"\n📌 Top 10 for {y_col} with 5 features")
    print(res_df.head(10))


📌 Top 10 for cap-color with 5 features
                              Feature Combination  Accuracy (%)
0  (class, cap-shape, cap-surface, bruises, odor)      37.27228

📌 Top 10 for ring-number with 5 features
                                 Feature Combination  Accuracy (%)
0  (class, cap-shape, cap-surface, cap-color, bru...     96.959626

📌 Top 10 for spore-print-color with 5 features
                                 Feature Combination  Accuracy (%)
0  (class, cap-shape, cap-surface, cap-color, bru...     69.374692


## Imputation with Best Features


In [ ]:
# 1) Pick best feature combo per target
best_features_map = {
    y_col: list(res_df.iloc[0]["Feature Combination"])
    for y_col, res_df in results.items()
    if res_df is not None and not res_df.empty
}

# 2) Convert to column indices
features_idx_map = {
    tgt: [df.columns.get_loc(c) for c in feats]
    for tgt, feats in best_features_map.items()
}

# 3) Run imputation with NumPy
imputed = data.copy().astype(np.float32)  # data is your numpy array
t0 = perf_counter()

for tgt, feats in features_idx_map.items():
    y_idx = df.columns.get_loc(tgt)
    imputed = knn_impute_numpy(imputed, y_idx, feats, k=3, batch_size=512)

best_features_map


{'cap-color': ['class', 'cap-shape', 'cap-surface', 'bruises', 'odor'],
 'ring-number': ['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises'],
 'spore-print-color': ['class',
  'cap-shape',
  'cap-surface',
  'cap-color',
  'bruises']}

## Check Remaining NaNs
Counts how many missing values are left in each target column after imputation.


In [ ]:
# Count remaining NaNs in target columns
remaining_nans = {
    c: int(np.isnan(imputed[:, df.columns.get_loc(c)]).sum())
    for c in best_features_map.keys()
}
print(remaining_nans)


{'cap-color': 0, 'ring-number': 0, 'spore-print-color': 0}


In [ ]:
# Convert back to Pandas and cast to int labels
df_imputed = pd.DataFrame(imputed, columns=df.columns).astype(int)
df_imputed.head()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
2,1,1,0,2,0,2,0,0,1,1,...,0,0,0,0,0,0,0,1,1,2
3,0,0,1,2,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,3,1,3,0,1,1,0,...,0,0,0,0,0,0,1,1,2,1


##  Measuring End Time
At the end of the pipeline, record the end time and calculate the total runtime



In [ ]:
end_time = time.time()
print(f"⏱️ [NumPy] Total execution time: {end_time - start_time:.2f} seconds")


⏱️ [NumPy] Total execution time: 2.91 seconds


# **PyTorch version**

## Measuring Runtime

In [ ]:
# Measuring Runtime
import time
start_time = time.time()

## Import Required Libraries (PyTorch Implementation)
Import core libraries (Pandas, NumPy, PyTorch) and detect the compute device (CPU or CUDA) for acceleration

In [ ]:
# Import Required Libraries (PyTorch Implementation)
import pandas as pd
import numpy as np
import torch
from time import perf_counter
import itertools
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Load and Encode Dataset

In [ ]:
# Read the mushroom dataset and convert all categorical columns to integer codes.
df = pd.read_csv("mushrooms.csv")

enc_mappings = {}
for col in df.columns:
    codes, uniques = pd.factorize(df[col], sort=False)
    df[col] = codes
    enc_mappings[col] = dict(zip(uniques, range(len(uniques))))

df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
2,1,1,0,2,0,2,0,0,1,1,...,0,0,0,0,0,0,0,1,1,2
3,0,0,1,2,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,3,1,3,0,1,1,0,...,0,0,0,0,0,0,1,1,2,1


## Convert to tensors and inject missing values
Move the encoded data into a PyTorch tensor (float dtype so NaNs are representable), define target columns with artificial NaNs, and prepare inputs for KNN.

In [ ]:
# Inject 10% NaNs into selected target columns and report NaN counts
data = torch.tensor(df.values, dtype=torch.float32, device=device)
n_rows, n_cols = data.shape

torch.manual_seed(42)
target_cols = ['cap-color', 'ring-number', 'spore-print-color']
target_idx = [df.columns.get_loc(c) for c in target_cols]

nan_frac = 0.10
for j in target_idx:
    idx = torch.randperm(n_rows, device=device)[:int(n_rows * nan_frac)]
    data[idx, j] = float('nan')

[(c, torch.isnan(data[:, df.columns.get_loc(c)]).sum().item()) for c in target_cols]

[('cap-color', 812), ('ring-number', 812), ('spore-print-color', 812)]

## KNN Imputation with PyTorch
Function to fill missing values in a target column using KNN over selected features, supporting GPU acceleration and batch processing.

In [ ]:
@torch.no_grad()
def knn_impute_torch(data, y_col_idx, x_col_indices, k=3, batch_size=512, device=None):
    """
    Impute missing labels in column y_col_idx using KNN over features x_col_indices.
    data: torch.FloatTensor [N, D] with NaNs allowed (float dtype). Operates in-place on a copy.
    Returns a new tensor with NaNs in y_col_idx imputed.
    """
    if device is None:
        device = data.device

    X = data[:, x_col_indices]
    y = data[:, y_col_idx]

    missing_mask = torch.isnan(y)
    keep_mask = ~missing_mask

    # nothing to impute
    if keep_mask.sum() == 0 or missing_mask.sum() == 0:
        return data.clone()

    train_X = X[keep_mask]
    train_y = y[keep_mask].to(torch.long)

    out = data.clone()

    # fill NaNs in train features with column means
    if torch.isnan(train_X).any():
        col_means = torch.nanmean(train_X, dim=0)
        nan_mask_tx = torch.isnan(train_X)
        train_X = torch.where(nan_mask_tx, col_means.expand_as(train_X), train_X)
    else:
        col_means = None

    miss_idx = torch.nonzero(missing_mask, as_tuple=False).squeeze(1)

    for start in range(0, miss_idx.numel(), batch_size):
        end = min(start + batch_size, miss_idx.numel())
        idx_batch = miss_idx[start:end]

        test_X = X[idx_batch]

        # fill NaNs in test features
        if torch.isnan(test_X).any():
            if col_means is None:
                col_means = torch.nanmean(train_X, dim=0)
            nan_mask_tx = torch.isnan(test_X)
            test_X = torch.where(nan_mask_tx, col_means.expand_as(test_X), test_X)

        # pairwise distances and KNN lookup
        dist = torch.cdist(test_X, train_X, p=2)

        vals, idxs = torch.topk(dist, k=min(k, dist.shape[1]), largest=False, dim=1)

        # majority vote among nearest neighbors
        nn_labels = train_y[idxs]
        preds, _ = torch.mode(nn_labels, dim=1)

        out[idx_batch, y_col_idx] = preds.to(torch.float32)

    return out

## KNN Feature Search & Evaluation with PyTorch
Sample feature combinations, run batched KNN classification or imputation with `torch.cdist`, compute accuracy per target, and use CUDA when available.


In [ ]:
def knn_accuracy_all_torch(df, target_cols, k=3, feature_sample_size=10, comb_size=3, device="cuda"):
    torch.set_grad_enabled(False)
    if device == "cuda" and not torch.cuda.is_available():
        print("⚠️ CUDA not available, falling back to CPU.")
        device = "cpu"

    results = {}

    for y_col in target_cols:
        non_missing = df[df[y_col].notna()]
        if non_missing.shape[0] <= 1:
            print(f"Skipping {y_col}: no known labels after NaN injection.")
            continue

        # choose candidate features excluding target
        x_candidates = [c for c in df.columns if c != y_col]
        x_candidates = x_candidates[:feature_sample_size]

        if len(x_candidates) < comb_size:
            print(f"Skipping {y_col}: not enough features for comb_size={comb_size}.")
            continue

        combos = list(itertools.combinations(x_candidates, comb_size))
        col_rows = []

        for combo in combos:
            combo = list(combo)

            # drop rows with NaNs in selected features
            nm = non_missing[non_missing[combo].notna().all(axis=1)]
            m = nm.shape[0]
            if m <= 1:
                continue

            # to Torch tensors
            X = torch.from_numpy(nm[combo].to_numpy(dtype=np.float32)).to(device)
            y = torch.from_numpy(nm[y_col].to_numpy()).to(device)

            # pairwise distance matrix (Euclidean)
            x2 = (X * X).sum(dim=1, keepdim=True)
            dists = x2 - 2 * (X @ X.T) + x2.T
            dists.fill_diagonal_(float("inf"))

            k_eff = min(k, m - 1)
            _, idx = torch.topk(dists, k=k_eff, dim=1, largest=False)

            # neighbor labels
            nn_labels = y[idx]

            # map labels to indices
            unique_labels, inv_idx = torch.unique(y, return_inverse=True)
            eq = nn_labels.unsqueeze(-1) == unique_labels.view(1, 1, -1)
            mapped = eq.float().argmax(dim=2)

            # majority vote
            L = unique_labels.shape[0]
            counts = torch.zeros((m, L), device=device, dtype=torch.int32)
            counts.scatter_add_(1, mapped, torch.ones_like(mapped, dtype=torch.int32))

            pred_idx = counts.argmax(dim=1)
            preds = unique_labels[pred_idx]

            # accuracy
            correct = (preds == y).sum().item()
            acc = 100.0 * correct / m
            col_rows.append((tuple(combo), float(acc)))

        # rank results for this target col
        if col_rows:
            col_rows.sort(key=lambda x: x[1], reverse=True)
            results[y_col] = pd.DataFrame(col_rows, columns=["Feature Combination", "Accuracy (%)"])

    return results

## Example Usage
Choose the feature set size to evaluate by editing comb_size.
The code includes examples for 3, 4, and 5 features, simply uncomment the block you need and run it (leave the others commented out), exactly the same as in the NumPy version

In [ ]:
# 10 best 3-feature combos
results = knn_accuracy_all_torch(df, target_cols, k=3, feature_sample_size=10, comb_size=3, device="cuda")
for y_col, res_df in results.items():
    print(f"\n📌 Top 10 for {y_col} with 3 features")
    print(res_df.head(10))

# 10 best 4-feature combos
# results = knn_accuracy_all_torch(df, target_cols, k=3, feature_sample_size=10, comb_size=4, device="cuda")
# for y_col, res_df in results.items():
#     print(f"\n📌 Top 10 for {y_col} with 4 features")
#     print(res_df.head(10))

# 10 best 5-feature combos
# results = knn_accuracy_all_torch(df, target_cols, k=3, feature_sample_size=10, comb_size=5, device="cuda")
# for y_col, res_df in results.items():
#     print(f"\n📌 Top 10 for {y_col} with 5 features")
#     print(res_df.head(10))


📌 Top 10 for cap-color with 3 features
                   Feature Combination  Accuracy (%)
0      (odor, gill-color, stalk-shape)     42.208272
1      (cap-surface, odor, gill-color)     41.765140
2  (odor, gill-attachment, gill-color)     41.358936
3       (cap-surface, odor, gill-size)     41.346627
4          (bruises, odor, gill-color)     41.297390
5     (odor, gill-spacing, gill-color)     41.149680
6        (cap-shape, odor, gill-color)     41.137371
7             (class, odor, gill-size)     41.075825
8      (odor, gill-spacing, gill-size)     40.866568
9            (class, odor, gill-color)     40.841950

📌 Top 10 for ring-number with 3 features
                     Feature Combination  Accuracy (%)
0        (cap-surface, odor, gill-color)     96.602659
1           (cap-shape, cap-color, odor)     96.528804
2     (cap-shape, cap-color, gill-color)     96.454948
3        (cap-shape, cap-color, bruises)     96.331856
4          (class, cap-shape, cap-color)     96.085672
5  (g

## Imputation with Best Features
Extract the top‑scoring feature combinations per target label and convert feature names to column indices for efficient tensor operations.

In [ ]:
# 1) Pick best feature combo per target
best_features_map = {
    y_col: list(res_df.iloc[0]["Feature Combination"])
    for y_col, res_df in results.items()
    if not res_df.empty
}

# 2) Convert to column indices
features_idx_map = {
    tgt: [df.columns.get_loc(c) for c in feats]
    for tgt, feats in best_features_map.items()
}

# 3) Run imputation with PyTorch
imputed = data.clone()
if device.type == "cuda":
    torch.cuda.synchronize()
t0 = perf_counter()

for tgt, feats in features_idx_map.items():
    y_idx = df.columns.get_loc(tgt)
    imputed = knn_impute_torch(imputed, y_idx, feats, k=3, batch_size=512, device=device)

if device.type == "cuda":
    torch.cuda.synchronize()
elapsed = perf_counter() - t0

# 4) Show runtime and chosen feature sets
device, elapsed
best_features_map


{'cap-color': ['odor', 'gill-color', 'stalk-shape'],
 'ring-number': ['cap-surface', 'odor', 'gill-color'],
 'spore-print-color': ['odor', 'gill-spacing', 'gill-color']}

## Check Remaining NaNs
Counts how many missing values are left in each target column after imputation.

In [ ]:
# Count remaining NaNs in target columns
remaining_nans = {c: int(torch.isnan(imputed[:, df.columns.get_loc(c)]).sum().item()) for c in best_features_map.keys()}
remaining_nans

{'cap-color': 0, 'ring-number': 0, 'spore-print-color': 0}

## Materialize imputed data to Pandas
Move tensors back to CPU, convert to a Pandas DataFrame, cast to integer labels, and preview the result.

In [ ]:
# Convert imputed tensor back to NumPy / DataFrame for inspection
imputed_cpu = imputed.detach().to('cpu').numpy()
df_imputed = pd.DataFrame(imputed_cpu, columns=df.columns).astype(int)
df_imputed.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
2,1,1,0,2,0,2,0,0,1,1,...,0,0,0,0,0,0,0,1,1,2
3,0,0,1,2,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,3,1,3,0,1,1,0,...,0,0,0,0,0,0,1,1,2,1


## Measuring End Time

In [ ]:
end_time = time.time()
print(f"⏱️ [PyTorch] Total execution time: {end_time - start_time:.2f} seconds")

⏱️ [PyTorch] Total execution time: 6.83 seconds
